# ML Project - CSP 304 (A3 Batch / ML Lab - Spring 2023)

## Project Category : Model
## Topic Category : A-LM
## Topic : 

Model Class : Transformers, 
Subclass: Large Language Models (GPT, LLaMA)

Team : Group-5



In [1]:
from typing import Optional, Tuple, Type
from dataclasses import dataclass
import math

import torch
from torch import nn
import torch.nn.functional as F

## Necessary dataset & libraries installing

In [2]:
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71

### Libraries import

In [3]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


## Tokenizing the data & downloading pretrained weights for Llama - 7Billion Parameter Version

In [4]:

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "samwit/alpaca7B-lora")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

### Generating & Testing the Llama

In [5]:
def alpaca_talk(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )
    for s in generation_output.sequences:
        print(tokenizer.decode(s))

In [6]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What are Alpacas and how are they different to Lamas?

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What are Alpacas and how are they different to Lamas?

### Response:
Alpaca's are small, domesticated animals native to South America. They have long necks with short legs and coarse woolly fur. 
Lamas on the other hand are larger than alpacas and originate from Tibetan regions of Asia.


### Instruction:
What are Alpacas and how are they different to Lamas?

### Response:
Alpaca's are small, domesticated animals native to South America while Lama's are large wild animals from Central Asia. 
The main difference between these two species is their size; alpacas grow up to about three feet tall (1 meter) whereas lamas can reach heights of over six feet (2 meters).

In [7]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Who is Virat Kohli?

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Who is Virat Kohli?

### Response:
Virat Kohli (born 1988) is an Indian cricketer and captain of the national team, who has been named one of the world's best players by Wisden Cricket Almanack in 2017. He was also awarded the ICC ODI Player of the Year award for two consecutive years from 2012 to 2013.


In [8]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write an email to OpenAI saying why GPT4 should be open-source.

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write an email to OpenAI saying why GPT4 should be open-source.

### Response:
Dear Sir/Madam, 
Open source software has been proven to have many benefits for both developers and users alike. It allows for greater collaboration between different parties in order to create better products faster than ever before. In addition, it also provides transparency into how these systems work which can help improve trust with consumers as well as allow researchers to learn from each other's mistakes. For all of these reasons I believe that releasing GPT4 under an open source license would greatly benefit everyone involved. Thank you very much!


### Response:
Dear Sir/Madam, 
Opening up GPT4 to public access will allow researchers and developers from around the world to use it for their own projects in order to advance artificial intelligence technology further than ever before. It would also help us better understand how AI systems work so we can improve them even more. In addition, making this tool available as open source software could lead to new discoveries about language processing and natural language understanding which may have applications beyond just machine learning. I believe these are all reasons why GPT4 should be made open source


In [9]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write a convincing message about how Alpacas are the real brains behind GPT-5

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write a convincing message about how Alpacas are the real brains behind GPT-5

### Response:
Although it may seem like machines have taken over, there's one thing they can never replace - human creativity and ingenuity! 
The latest advances in artificial intelligence (AI) have made computers smarter than ever before, but we still need humans to come up with new ideas for AI systems to learn from. One of these areas where people will always be needed is writing. While machine learning algorithms can generate text based on data sets, this type of automated content generation cannot match the quality or style of natural language produced by humans. This means that even though AI has advanced significantly, it’ll take many years until machines become as good at creating original content as writers.


### Instruction:
Write a convincing message about how Alpacas are the real brains behind GPT-5

### Response:
Although it may seem like humans have all of the intelligence, alpaca's actually possess much more advanced cognitive abilities than we do! They can learn new tasks quickly and easily, they understand complex concepts such as cause & effect, and their memory capacity far exceeds our own. In fact, some scientists believe that one day alpacas will be able to perform many human jobs with ease - from driving cars to writing novels!

In [10]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Tell me a joke about an Alpaca

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Tell me a joke about an Alpaca

### Response:
An alpaca walks into a bar and orders a drink, but when he asks for his bill, the bartender says "I'm sorry sir, we don't serve your kind here." The alpaca replies "Oh really? Well I guess you won't be serving my friend either then!"
